<a href="https://colab.research.google.com/github/Kaif10/ML-ALgorithms-from-scratch/blob/master/KNN_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook I have coded the KNN algorithm from scratch and then compared my model's accurcay on the smae data set ith Scikit learn's implementation.

.

In [0]:
import pandas as pd
import numpy as np

In [0]:
data_2C = pd.read_csv("column_2C_weka.csv")

In [0]:
data_2C.head()

,pelvic_incidence,pelvic_tilt numeric,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
0,63.027818,22.552586,39.609117,40.475232,98.672917,-0.254400,Abnormal
1,39.056951,10.060991,25.015378,28.995960,114.405425,4.564259,Abnormal
2,68.832021,22.218482,50.092194,46.613539,105.985135,-3.530317,Abnormal
3,69.297008,24.652878,44.311238,44.644130,101.868495,11.211523,Abnormal
4,49.712859,9.652075,28.317406,40.060784,108.168725,7.918501,Abnormal


In [0]:
#analyse our data
data_2C.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
pelvic_incidence,310.0,60.496653,17.236520,26.147921,46.430294,58.691038,72.877696,129.834041
pelvic_tilt numeric,310.0,17.542822,10.008330,-6.554948,10.667069,16.357689,22.120395,49.431864
lumbar_lordosis_angle,310.0,51.930930,18.554064,14.000000,37.000000,49.562398,63.000000,125.742385
sacral_slope,310.0,42.953831,13.423102,13.366931,33.347122,42.404912,52.695888,121.429566
pelvic_radius,310.0,117.920655,13.317377,70.082575,110.709196,118.268178,125.467674,163.071041
degree_spondylolisthesis,310.0,26.296694,37.559027,-11.058179,1.603727,11.767934,41.287352,418.543082


In [0]:
data_2C.dtypes

pelvic_incidence            float64
pelvic_tilt numeric         float64
lumbar_lordosis_angle       float64
sacral_slope                float64
pelvic_radius               float64
degree_spondylolisthesis    float64
class                        object
dtype: object

In [0]:
colnames_numeric = data_2C.columns[0:6]

Scaling our features column.

In [0]:
#Scaling a data in always a good idea while using KNN
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data_2C[colnames_numeric] = scaler.fit_transform(data_2C[colnames_numeric])

In [0]:
data_2C.head()

,pelvic_incidence,pelvic_tilt numeric,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
0,0.355688,0.519900,0.229180,0.250857,0.307461,0.025148,Abnormal
1,0.124501,0.296783,0.098578,0.144629,0.476649,0.036365,Abnormal
2,0.411666,0.513932,0.322995,0.307661,0.386097,0.017523,Abnormal
3,0.416151,0.557414,0.271260,0.289436,0.341826,0.051838,Abnormal
4,0.227272,0.289479,0.128129,0.247022,0.409579,0.044173,Abnormal


In [0]:
#Convert dataframe to list
df = data_2C.values.tolist()

In [0]:
#Breaking the data into training and test set
import random
def train_test_split(data, split, trainingSet = [], testSet = []):
    for x in range(len(data)):
        if random.random() < split:
            trainingSet.append(data[x])
        else:
            testSet.append(data[x])

In [0]:
#split 
trainingSet = []
testSet = []
split = 0.66
train_test_split(df, split, trainingSet, testSet)

In [0]:
print(len(trainingSet))
print(len(testSet))

100

In [0]:
#Define Euclidean distances
import math
def Euclideandist(x,xi, length):
    d = 0.0
    for i in range(length):
        d += pow(float(x[i])- float(xi[i]),2)
    return math.sqrt(d)

In [0]:
#Getting the K neighbours having the closest Euclidean distance to the test instance
import operator
def getNeighbors(trainingSet, testInstance, k):
    distances = []
    length = len(testInstance)-1
    for x in range(len(trainingSet)):
        dist = Euclideandist(testInstance, trainingSet[x], length)
        distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

In [0]:
#After sorting the neighbours based on their respective classes, max voting to give the final class of the test instance
import operator
def getResponse(neighbors):
	classVotes = {}
	for x in range(len(neighbors)):
		response = neighbors[x][-1]
		if response in classVotes:
			classVotes[response] += 1
		else:
			classVotes[response] = 1
	sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)#Sorting it based on votes
	return sortedVotes[0][0] #Please note we need the class for the top voted class, hence [0][0]#

In [0]:
#Getting the accuracy
def getAccuracy(testSet, predictions):
	correct = 0
	for x in range(len(testSet)):
		if testSet[x][-1] == predictions[x]:
			correct += 1
	return (correct/float(len(testSet))) * 100.0

In [0]:
# generate predictions
predictions=[]
k = 3
for x in range(len(testSet)):
    neighbors = getNeighbors(trainingSet, testSet[x], k)
    result = getResponse(neighbors)
    predictions.append(result)
    print('> predicted=' + repr(result) + ', actual=' + repr(testSet[x][-1]))

> predicted='Abnormal', actual='Abnormal'
> predicted='Abnormal', actual='Abnormal'
> predicted='Normal', actual='Abnormal'
> predicted='Abnormal', actual='Abnormal'
> predicted='Abnormal', actual='Abnormal'
> predicted='Abnormal', actual='Abnormal'
> predicted='Abnormal', actual='Abnormal'
> predicted='Normal', actual='Abnormal'
> predicted='Abnormal', actual='Abnormal'
> predicted='Normal', actual='Abnormal'
> predicted='Abnormal', actual='Abnormal'
> predicted='Abnormal', actual='Abnormal'
> predicted='Abnormal', actual='Abnormal'
> predicted='Normal', actual='Abnormal'
> predicted='Abnormal', actual='Abnormal'
> predicted='Abnormal', actual='Abnormal'
> predicted='Abnormal', actual='Abnormal'
> predicted='Abnormal', actual='Abnormal'
> predicted='Normal', actual='Abnormal'
> predicted='Normal', actual='Abnormal'
> predicted='Abnormal', actual='Abnormal'
> predicted='Normal', actual='Abnormal'
> predicted='Normal', actual='Abnormal'
> predicted='Abnormal', actual='Abnormal'
> predic

In [0]:
accuracy = getAccuracy(testSet, predictions)
print('Accuracy: ' + repr(accuracy) + '%')

Accuracy: 80.0%


.

Scikit learn's implementaion

In [0]:
#Implementing Naive Bayes using scikitlearn
trainingSet2 = pd.DataFrame(np.array(trainingSet).reshape(len(trainingSet),7), columns = data_2C.columns)
testSet2 = pd.DataFrame(np.array(testSet).reshape(len(testSet),7), columns = data_2C.columns)

In [0]:
trainingSet2.head()

,pelvic_incidence,pelvic_tilt numeric,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
0,0.3556878816384217,0.5198998354211468,0.22917997405738216,0.25085730007495866,0.30746116406185287,0.025148386796833937,Abnormal
1,0.1245010390093278,0.29678310371966715,0.09857833418098988,0.1446293515280553,0.47664890731280163,0.03636497080423512,Abnormal
2,0.4161510431394515,0.5574138832291958,0.27125998782261546,0.2894358396770871,0.34182648376044333,0.051838074359006905,Abnormal
3,0.2631500431605651,0.4004386691677185,0.20731554786791268,0.2239595707840731,0.542916243348394,0.03968035226861981,Abnormal
4,0.18535588331531994,0.3091899534016523,0.1345805254891395,0.19659164797951825,0.5074553313169391,0.0008899131937454424,Abnormal


In [0]:
trainingSet2.dtypes

pelvic_incidence            object
pelvic_tilt numeric         object
lumbar_lordosis_angle       object
sacral_slope                object
pelvic_radius               object
degree_spondylolisthesis    object
class                       object
dtype: object

In [0]:
trainingSet2[colnames_numeric] = trainingSet2[colnames_numeric].apply(pd.to_numeric, errors = 'coerce', axis = 0)

In [0]:
trainingSet2.dtypes

pelvic_incidence            float64
pelvic_tilt numeric         float64
lumbar_lordosis_angle       float64
sacral_slope                float64
pelvic_radius               float64
degree_spondylolisthesis    float64
class                        object
dtype: object

In [0]:
testSet2[colnames_numeric] = testSet2[colnames_numeric].apply(pd.to_numeric, errors = 'coerce', axis = 0)
testSet2.dtypes

pelvic_incidence            float64
pelvic_tilt numeric         float64
lumbar_lordosis_angle       float64
sacral_slope                float64
pelvic_radius               float64
degree_spondylolisthesis    float64
class                        object
dtype: object

In [0]:
#here we don't need  to convert dataframe to list for feeding data into scikit learn's model
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 3)
x_train,y_train = trainingSet2.loc[:,trainingSet2.columns != 'class'], trainingSet2.loc[:,'class']
x_test,y_test = testSet2.loc[:,testSet2.columns != 'class'], testSet2.loc[:,'class']


In [0]:
knn.fit(x_train,y_train)
prediction = knn.predict(x_test)
print('Prediction: {}'.format(prediction))
print('With KNN (K=3) accuracy is: ',knn.score(x_test,y_test)) # accuracy

Prediction: ['Abnormal' 'Abnormal' 'Normal' 'Abnormal' 'Abnormal' 'Abnormal'
 'Abnormal' 'Normal' 'Abnormal' 'Normal' 'Abnormal' 'Abnormal' 'Abnormal'
 'Normal' 'Abnormal' 'Abnormal' 'Abnormal' 'Abnormal' 'Normal' 'Normal'
 'Abnormal' 'Normal' 'Normal' 'Abnormal' 'Abnormal' 'Abnormal' 'Abnormal'
 'Normal' 'Abnormal' 'Abnormal' 'Abnormal' 'Normal' 'Abnormal' 'Abnormal'
 'Abnormal' 'Normal' 'Normal' 'Abnormal' 'Abnormal' 'Abnormal' 'Abnormal'
 'Abnormal' 'Abnormal' 'Abnormal' 'Abnormal' 'Abnormal' 'Abnormal'
 'Abnormal' 'Abnormal' 'Abnormal' 'Abnormal' 'Abnormal' 'Normal'
 'Abnormal' 'Abnormal' 'Abnormal' 'Abnormal' 'Abnormal' 'Abnormal'
 'Abnormal' 'Abnormal' 'Abnormal' 'Abnormal' 'Abnormal' 'Abnormal'
 'Abnormal' 'Abnormal' 'Abnormal' 'Abnormal' 'Abnormal' 'Normal' 'Normal'
 'Abnormal' 'Normal' 'Normal' 'Normal' 'Normal' 'Normal' 'Normal' 'Normal'
 'Abnormal' 'Abnormal' 'Normal' 'Abnormal' 'Abnormal' 'Normal' 'Normal'
 'Normal' 'Normal' 'Normal' 'Normal' 'Normal' 'Normal' 'Normal' 'Nor

Accuracy of 1st model: 80%

Accurcay of 2nd model: 0.8 i.e around  80%

---

